In [93]:
import pandas as pd
import numpy as np
from statistics import geometric_mean

import seaborn as sns
import matplotlib.pyplot as plt

from collections import OrderedDict
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.preprocessing import StandardScaler
from sklearn.utils import check_X_y

from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.dummy import DummyClassifier

from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
pd.set_option('display.max_columns', None)

In [94]:
set_1 = pd.read_csv("analysis_set_1.csv")
set_1 = set_1.filter(['NAME', 'RANDOM SEED'], axis = 1)
ml = pd.read_csv('ml.csv')
analysis_ml = set_1.merge(ml, on = ['NAME', 'RANDOM SEED'], how = 'left')
analysis_ml = analysis_ml.drop(['Unnamed: 0', 'Time Label', 'Nodes Label'], axis = 1)
# analysis_ml.to_csv('Set_1_analysis.csv')
analysis_ml

,NAME,RANDOM SEED,TIME W/ CUTS,NODES W/ CUTS,TIME W/O CUTS,NODES W/O CUTS,GAP W/ CUTS,GAP W/O CUTS,INITIAL LP,PRESOLVING TIME,vars,bin_vars,int_vars,cont_vars,constr,linsetpart_constr,linsetpack_constr,linsetcov_constr,lincard_constr,lineqknap_constr,linbinpack_constr,linknaps_constr,linintknap_constr,linmixbin_constr,nonzeroes,obj_dyn,dynamism_mean,presol_vars,presol_int_vars,presol_constr
0,10teams.mps.gz,2,11.088517,30,12.212530,276,0.0,0.0,20.0,0.062201,7.613325,0.888889,0.000000,0.111111,5.438079,0.347826,0.173913,0.0,0.000000,0.0,0.0,0.0,0.0,0.478261,0.026087,1.63347,0.0,7.377759,0.000000,5.347108
1,10teams.mps.gz,4,11.186637,30,12.444448,276,0.0,0.0,20.0,0.060087,7.613325,0.888889,0.000000,0.111111,5.438079,0.347826,0.173913,0.0,0.000000,0.0,0.0,0.0,0.0,0.478261,0.026087,1.63347,0.0,7.377759,0.000000,5.347108
2,10teams.mps.gz,8,11.044666,30,12.331489,276,0.0,0.0,20.0,0.058780,7.613325,0.888889,0.000000,0.111111,5.438079,0.347826,0.173913,0.0,0.000000,0.0,0.0,0.0,0.0,0.478261,0.026087,1.63347,0.0,7.377759,0.000000,5.347108
3,10teams.mps.gz,16,10.835616,30,12.467186,276,0.0,0.0,20.0,0.059116,7.613325,0.888889,0.000000,0.111111,5.438079,0.347826,0.173913,0.0,0.000000,0.0,0.0,0.0,0.0,0.478261,0.026087,1.63347,0.0,7.377759,0.000000,5.347108
4,10teams.mps.gz,32,10.833424,30,12.477946,276,0.0,0.0,20.0,0.058801,7.613325,0.888889,0.000000,0.111111,5.438079,0.347826,0.173913,0.0,0.000000,0.0,0.0,0.0,0.0,0.478261,0.026087,1.63347,0.0,7.377759,0.000000,5.347108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1770,wachplan.mps.gz,2,1427.930347,67520,1801.604630,98056,0.0,0.0,-24.0,1.552340,8.119994,0.999702,0.000298,0.000000,7.347944,0.000000,0.000000,0.0,0.144237,0.0,0.0,0.0,0.0,0.018030,0.017120,0.00000,0.0,7.741534,0.000434,6.582025
1771,wachplan.mps.gz,4,1444.662760,67520,1804.842432,98056,0.0,0.0,-24.0,1.914422,8.119994,0.999702,0.000298,0.000000,7.347944,0.000000,0.000000,0.0,0.144237,0.0,0.0,0.0,0.0,0.018030,0.017120,0.00000,0.0,7.741534,0.000434,6.582025
1772,wachplan.mps.gz,8,1431.659624,67520,1737.152014,98056,0.0,0.0,-24.0,1.507830,8.119994,0.999702,0.000298,0.000000,7.347944,0.000000,0.000000,0.0,0.144237,0.0,0.0,0.0,0.0,0.018030,0.017120,0.00000,0.0,7.741534,0.000434,6.582025
1773,wachplan.mps.gz,16,1433.459125,67520,1758.587956,98056,0.0,0.0,-24.0,1.547767,8.119994,0.999702,0.000298,0.000000,7.347944,0.000000,0.000000,0.0,0.144237,0.0,0.0,0.0,0.0,0.018030,0.017120,0.00000,0.0,7.741534,0.000434,6.582025


In [16]:
analysis_ml = analysis_ml.groupby('NAME').mean()
analysis_ml = analysis_ml.reset_index()
#feat.to_csv('unique.csv')
analysis_ml

,NAME,RANDOM SEED,TIME W/ CUTS,NODES W/ CUTS,TIME W/O CUTS,NODES W/O CUTS,GAP W/ CUTS,GAP W/O CUTS,INITIAL LP,PRESOLVING TIME,vars,bin_vars,int_vars,cont_vars,constr,linsetpart_constr,linsetpack_constr,linsetcov_constr,lincard_constr,lineqknap_constr,linbinpack_constr,linknaps_constr,linintknap_constr,linmixbin_constr,nonzeroes,obj_dyn,dynamism_mean,presol_vars,presol_int_vars,presol_constr
0,10teams.mps.gz,12.4,10.997772,30.0,12.386720,276.0,0.0,0.0,2.000000e+01,0.059797,7.613325,0.888889,0.000000,0.111111,5.438079,0.347826,0.173913,0.000000,0.000000,0.0,0.0,0.0,0.0,0.478261,0.026087,1.63347,0.000000,7.377759,0.000000,5.347108
1,22433.mps.gz,12.4,2.535930,1.0,0.650582,5.0,0.0,0.0,2.124053e+04,0.025466,6.061457,0.538462,0.000000,0.461538,5.288267,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.000000,0.040121,0.00000,0.015269,6.061457,0.000000,5.288267
2,23588.mps.gz,12.4,5.527370,576.0,2.399748,981.0,0.0,0.0,7.649866e+03,0.031946,5.908083,0.627717,0.000000,0.372283,4.919981,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.000000,0.073409,0.00000,0.026683,5.908083,0.000000,4.919981
3,30_70_45_05_100.mps.gz,12.4,282.653795,46.0,240.079782,201.0,0.0,0.0,-5.336000e+03,1.625184,9.284705,0.999907,0.000000,0.000093,9.396820,0.000000,0.000000,0.004066,0.000000,0.0,0.0,0.0,0.0,0.149627,0.000354,3.72884,0.000000,9.284334,0.000000,9.396654
4,30_70_45_095_100.mps.gz,12.4,743.997969,201.0,55.714462,1.0,0.0,0.0,-6.132000e+03,2.143665,9.303466,0.999909,0.000000,0.000091,9.435562,0.000000,0.000000,0.001437,0.000000,0.0,0.0,0.0,0.0,0.164139,0.000339,3.78902,0.000000,9.303375,0.000000,9.435562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,triptim1.mps.gz,12.4,2446.382658,78.0,690.691198,11.0,0.0,0.0,-2.034000e+00,5.952476,10.310784,0.680619,0.319148,0.000233,9.661798,0.000000,0.130460,0.257099,0.000000,0.0,0.0,0.0,0.0,0.000000,0.001092,0.00000,0.002040,10.153818,0.372767,9.661352
351,umts.mps.gz,12.4,1079.840992,144227.0,11604.852368,3106370.0,0.0,0.0,1.460426e+06,0.230250,7.988543,0.950797,0.024432,0.024771,8.404024,0.033595,0.000000,0.403359,0.002688,0.0,0.0,0.0,0.0,0.012094,0.001738,6.14613,0.027856,7.411556,0.039879,7.541152
352,unitcal_7.mps.gz,12.4,940.799926,705.0,10563.733016,196989.0,0.0,0.0,1.028415e+06,6.065447,10.156384,0.110891,0.000000,0.889109,10.798330,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.356669,0.000101,4.30103,0.795567,10.083139,0.000000,10.642349
353,var-smallemery-m6j6.mps.gz,12.4,7171.038360,685889.0,10246.224785,836578.0,0.0,0.0,-1.697219e+03,8.342061,8.631949,0.999643,0.000000,0.000357,9.504203,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.011181,0.011306,0.00000,0.039372,8.113127,0.000000,9.259321


In [19]:
time_label = []
for index, row in analysis_ml.iterrows():
    if row['TIME W/ CUTS'] > row['TIME W/O CUTS']:
        time_label.append(-1)
    elif row['TIME W/ CUTS'] <= row['TIME W/O CUTS']:
        time_label.append(1)

analysis_ml['Time Label'] = time_label

analysis_ml

,NAME,RANDOM SEED,TIME W/ CUTS,NODES W/ CUTS,TIME W/O CUTS,NODES W/O CUTS,GAP W/ CUTS,GAP W/O CUTS,INITIAL LP,PRESOLVING TIME,vars,bin_vars,int_vars,cont_vars,constr,linsetpart_constr,linsetpack_constr,linsetcov_constr,lincard_constr,lineqknap_constr,linbinpack_constr,linknaps_constr,linintknap_constr,linmixbin_constr,nonzeroes,obj_dyn,dynamism_mean,presol_vars,presol_int_vars,presol_constr,Time Label
0,10teams.mps.gz,12.4,10.997772,30.0,12.386720,276.0,0.0,0.0,2.000000e+01,0.059797,7.613325,0.888889,0.000000,0.111111,5.438079,0.347826,0.173913,0.000000,0.000000,0.0,0.0,0.0,0.0,0.478261,0.026087,1.63347,0.000000,7.377759,0.000000,5.347108,1
1,22433.mps.gz,12.4,2.535930,1.0,0.650582,5.0,0.0,0.0,2.124053e+04,0.025466,6.061457,0.538462,0.000000,0.461538,5.288267,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.000000,0.040121,0.00000,0.015269,6.061457,0.000000,5.288267,-1
2,23588.mps.gz,12.4,5.527370,576.0,2.399748,981.0,0.0,0.0,7.649866e+03,0.031946,5.908083,0.627717,0.000000,0.372283,4.919981,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.000000,0.073409,0.00000,0.026683,5.908083,0.000000,4.919981,-1
3,30_70_45_05_100.mps.gz,12.4,282.653795,46.0,240.079782,201.0,0.0,0.0,-5.336000e+03,1.625184,9.284705,0.999907,0.000000,0.000093,9.396820,0.000000,0.000000,0.004066,0.000000,0.0,0.0,0.0,0.0,0.149627,0.000354,3.72884,0.000000,9.284334,0.000000,9.396654,-1
4,30_70_45_095_100.mps.gz,12.4,743.997969,201.0,55.714462,1.0,0.0,0.0,-6.132000e+03,2.143665,9.303466,0.999909,0.000000,0.000091,9.435562,0.000000,0.000000,0.001437,0.000000,0.0,0.0,0.0,0.0,0.164139,0.000339,3.78902,0.000000,9.303375,0.000000,9.435562,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,triptim1.mps.gz,12.4,2446.382658,78.0,690.691198,11.0,0.0,0.0,-2.034000e+00,5.952476,10.310784,0.680619,0.319148,0.000233,9.661798,0.000000,0.130460,0.257099,0.000000,0.0,0.0,0.0,0.0,0.000000,0.001092,0.00000,0.002040,10.153818,0.372767,9.661352,-1
351,umts.mps.gz,12.4,1079.840992,144227.0,11604.852368,3106370.0,0.0,0.0,1.460426e+06,0.230250,7.988543,0.950797,0.024432,0.024771,8.404024,0.033595,0.000000,0.403359,0.002688,0.0,0.0,0.0,0.0,0.012094,0.001738,6.14613,0.027856,7.411556,0.039879,7.541152,1
352,unitcal_7.mps.gz,12.4,940.799926,705.0,10563.733016,196989.0,0.0,0.0,1.028415e+06,6.065447,10.156384,0.110891,0.000000,0.889109,10.798330,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.356669,0.000101,4.30103,0.795567,10.083139,0.000000,10.642349,1
353,var-smallemery-m6j6.mps.gz,12.4,7171.038360,685889.0,10246.224785,836578.0,0.0,0.0,-1.697219e+03,8.342061,8.631949,0.999643,0.000000,0.000357,9.504203,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.011181,0.011306,0.00000,0.039372,8.113127,0.000000,9.259321,1


In [21]:
X = analysis_ml.iloc[0:, 0:-1]
y = analysis_ml.iloc[0:, -1]
X

,NAME,RANDOM SEED,TIME W/ CUTS,NODES W/ CUTS,TIME W/O CUTS,NODES W/O CUTS,GAP W/ CUTS,GAP W/O CUTS,INITIAL LP,PRESOLVING TIME,vars,bin_vars,int_vars,cont_vars,constr,linsetpart_constr,linsetpack_constr,linsetcov_constr,lincard_constr,lineqknap_constr,linbinpack_constr,linknaps_constr,linintknap_constr,linmixbin_constr,nonzeroes,obj_dyn,dynamism_mean,presol_vars,presol_int_vars,presol_constr
0,10teams.mps.gz,12.4,10.997772,30.0,12.386720,276.0,0.0,0.0,2.000000e+01,0.059797,7.613325,0.888889,0.000000,0.111111,5.438079,0.347826,0.173913,0.000000,0.000000,0.0,0.0,0.0,0.0,0.478261,0.026087,1.63347,0.000000,7.377759,0.000000,5.347108
1,22433.mps.gz,12.4,2.535930,1.0,0.650582,5.0,0.0,0.0,2.124053e+04,0.025466,6.061457,0.538462,0.000000,0.461538,5.288267,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.000000,0.040121,0.00000,0.015269,6.061457,0.000000,5.288267
2,23588.mps.gz,12.4,5.527370,576.0,2.399748,981.0,0.0,0.0,7.649866e+03,0.031946,5.908083,0.627717,0.000000,0.372283,4.919981,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.000000,0.073409,0.00000,0.026683,5.908083,0.000000,4.919981
3,30_70_45_05_100.mps.gz,12.4,282.653795,46.0,240.079782,201.0,0.0,0.0,-5.336000e+03,1.625184,9.284705,0.999907,0.000000,0.000093,9.396820,0.000000,0.000000,0.004066,0.000000,0.0,0.0,0.0,0.0,0.149627,0.000354,3.72884,0.000000,9.284334,0.000000,9.396654
4,30_70_45_095_100.mps.gz,12.4,743.997969,201.0,55.714462,1.0,0.0,0.0,-6.132000e+03,2.143665,9.303466,0.999909,0.000000,0.000091,9.435562,0.000000,0.000000,0.001437,0.000000,0.0,0.0,0.0,0.0,0.164139,0.000339,3.78902,0.000000,9.303375,0.000000,9.435562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,triptim1.mps.gz,12.4,2446.382658,78.0,690.691198,11.0,0.0,0.0,-2.034000e+00,5.952476,10.310784,0.680619,0.319148,0.000233,9.661798,0.000000,0.130460,0.257099,0.000000,0.0,0.0,0.0,0.0,0.000000,0.001092,0.00000,0.002040,10.153818,0.372767,9.661352
351,umts.mps.gz,12.4,1079.840992,144227.0,11604.852368,3106370.0,0.0,0.0,1.460426e+06,0.230250,7.988543,0.950797,0.024432,0.024771,8.404024,0.033595,0.000000,0.403359,0.002688,0.0,0.0,0.0,0.0,0.012094,0.001738,6.14613,0.027856,7.411556,0.039879,7.541152
352,unitcal_7.mps.gz,12.4,940.799926,705.0,10563.733016,196989.0,0.0,0.0,1.028415e+06,6.065447,10.156384,0.110891,0.000000,0.889109,10.798330,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.356669,0.000101,4.30103,0.795567,10.083139,0.000000,10.642349
353,var-smallemery-m6j6.mps.gz,12.4,7171.038360,685889.0,10246.224785,836578.0,0.0,0.0,-1.697219e+03,8.342061,8.631949,0.999643,0.000000,0.000357,9.504203,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.011181,0.011306,0.00000,0.039372,8.113127,0.000000,9.259321


In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=0)
X_train = X_train.drop(['NAME', 'RANDOM SEED', 'TIME W/ CUTS', 'NODES W/ CUTS', 
                        'TIME W/O CUTS', 'NODES W/O CUTS', 'GAP W/ CUTS', 'GAP W/O CUTS'], axis = 1)
cols = X_train.copy()
X_test = X_test.drop(['NAME', 'RANDOM SEED', 'TIME W/ CUTS', 'NODES W/ CUTS', 
                      'TIME W/O CUTS', 'NODES W/O CUTS', 'GAP W/ CUTS', 'GAP W/O CUTS'], axis = 1)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
cols

(284, 22) (284,)
(71, 22) (71,)


,INITIAL LP,PRESOLVING TIME,vars,bin_vars,int_vars,cont_vars,constr,linsetpart_constr,linsetpack_constr,linsetcov_constr,lincard_constr,lineqknap_constr,linbinpack_constr,linknaps_constr,linintknap_constr,linmixbin_constr,nonzeroes,obj_dyn,dynamism_mean,presol_vars,presol_int_vars,presol_constr
298,-9.200187e+03,0.042668,6.733402,0.057143,0.000000,0.942857,7.083388,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.114094,0.003428,0.827942,0.234359,6.733402,0.000000,7.083388
247,-2.492000e+03,1.474001,10.386438,0.883395,0.000000,0.116605,9.522081,0.000000,0.392224,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.237443,0.000423,3.000000,0.000000,9.092345,0.000000,9.239025
309,1.324444e+01,0.026762,6.133398,0.976139,0.000000,0.023861,4.219508,0.000000,0.000000,0.000000,0.147059,0.0,0.000000,0.000000,0.000000,0.661765,0.026668,0.000000,0.758467,5.888878,0.000000,3.970292
336,0.000000e+00,0.307568,8.058960,1.000000,0.000000,0.000000,9.158205,0.000000,0.000000,0.647282,0.000000,0.0,0.000000,0.000000,0.000000,0.244627,0.001267,0.000000,0.213065,7.667626,0.000000,8.767173
142,-2.599800e+04,19.519242,9.234057,0.975488,0.024512,0.000000,9.158942,0.112959,0.605853,0.002948,0.232446,0.0,0.000000,0.000105,0.000000,0.000105,0.001384,2.397940,0.054545,9.234057,0.019336,9.158942
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,4.442776e+08,1.902435,9.295967,1.000000,0.000000,0.000000,6.715383,0.000000,0.756364,0.020606,0.000000,0.0,0.000000,0.000000,0.000000,0.219394,0.035195,1.531250,0.098539,9.295967,0.000000,6.715383
192,0.000000e+00,15.067371,10.619814,0.127656,0.000000,0.872344,11.617520,0.002072,0.000486,0.000000,0.000000,0.0,0.000108,0.000054,0.000000,0.350567,0.000121,2.595280,0.018692,10.619814,0.000000,11.617520
117,3.000000e+00,1.154996,8.974745,0.970130,0.029870,0.000000,9.558388,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.094330,0.000000,0.000737,0.000000,2.724850,8.023225,0.037365,8.583917
47,3.000000e+00,0.309220,9.128696,1.000000,0.000000,0.000000,9.041567,0.272641,0.727359,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000434,0.000000,0.000000,9.128696,0.000000,9.041567


In [23]:
scaler = StandardScaler()
# Fit only to the training data
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [24]:
cols = cols.iloc[0:, 0:]

bestfeatures = SelectKBest(score_func=f_classif, k = 'all')
fit = bestfeatures.fit(X_train,y_train)
scores = pd.DataFrame(fit.scores_)
columns = pd.DataFrame(cols.columns)
featureScores = pd.concat([columns,scores],axis=1)
featureScores
featureScores.columns = ['Features','Score']  #naming the dataframe columns
print(featureScores.nlargest(22,'Score')) #print best features
data=featureScores.nlargest(22,'Score')

             Features     Score
17            obj_dyn  3.702554
12  linbinpack_constr  3.472830
16          nonzeroes  2.962461
1     PRESOLVING TIME  2.455590
3            bin_vars  2.061848
9    linsetcov_constr  2.054446
15   linmixbin_constr  2.007376
10     lincard_constr  1.604994
5           cont_vars  1.374379
11   lineqknap_constr  0.968273
4            int_vars  0.832279
0          INITIAL LP  0.743893
7   linsetpart_constr  0.652478
19        presol_vars  0.632712
14  linintknap_constr  0.580834
2                vars  0.527535
21      presol_constr  0.317150
6              constr  0.240116
20    presol_int_vars  0.149405
13    linknaps_constr  0.074020
8   linsetpack_constr  0.057341
18      dynamism_mean  0.000006


In [25]:
X = analysis_ml.drop(['Time Label', 'dynamism_mean', 'linsetpack_constr',
              'linknaps_constr', 'presol_int_vars', 'constr', 'presol_constr',
              'vars'], axis = 1)

y = analysis_ml.iloc[0:, -1]

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=0)

test_df = X_test.copy()
train_df = X_train.copy()

X_train = X_train.drop(['NAME', 'RANDOM SEED', 'TIME W/ CUTS', 'NODES W/ CUTS', 
                        'TIME W/O CUTS', 'NODES W/O CUTS', 'GAP W/ CUTS', 'GAP W/O CUTS'], axis = 1)
X_test = X_test.drop(['NAME', 'RANDOM SEED', 'TIME W/ CUTS', 'NODES W/ CUTS', 
                        'TIME W/O CUTS', 'NODES W/O CUTS', 'GAP W/ CUTS', 'GAP W/O CUTS'], axis = 1)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(284, 15) (284,)
(71, 15) (71,)


In [27]:
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [28]:
model_params={'svm':{'model': SVC(kernel = 'rbf', random_state = 0), 
                    'params':{'C':[10, 100, 1000], 'gamma': [1e-2, 1e-3, 1e-4]
                             }
                    },
              'rfm':{'model': RandomForestClassifier(random_state = 0), 
                    'params':{'n_estimators':[100, 200, 500], 
                             'max_depth':[5, 10, 15]
                             }
                    },
              'lrm':{'model': LogisticRegression(solver='lbfgs', random_state = 0), 
                    'params':{'C':[10, 100, 1000], 
                              'max_iter':[100, 500, 1000]
                             }
                    },
              'et':{'model': ExtraTreesClassifier(random_state=0), 
                    'params':{'n_estimators': [100, 200, 500], 
                              'max_depth': [5, 10, 15]
                             }
                    },
              'nn':{'model': MLPClassifier(solver='lbfgs', batch_size='auto', learning_rate='constant', 
                                           power_t=0.5, max_iter=500, shuffle=True, tol=0.00001, verbose=True, 
                                           early_stopping=False, validation_fraction=0.1, random_state=0),
                    'params':{'hidden_layer_sizes': [(5, ), (10, ), (15, )],
                                 'learning_rate_init': [1e-1, 1e-2, 1e-3],
                                 'alpha':  [1e-2, 1e-3, 1e-4],
                                 'activation': ['tanh', 'relu', 'logistic']
                             }
                    }
             }
scores=[]
for model_name, mp in model_params.items():
    clf=GridSearchCV(mp['model'], mp['params'], cv = None, return_train_score = False, n_jobs = -1)
    clf.fit(X_train, y_train)
    scores.append({'model': model_name, 'best_score':clf.best_score_, 'best_params':clf.best_params_})

In [30]:
scores=pd.DataFrame(scores).sort_values(by = ['best_score'], ascending = False)
scores.to_csv('set_1_scores.csv')

In [31]:
scores = OrderedDict()
target_names = ['-1', '1']

print('SUPPORT VECTOR MACHINES')
clf = SVC(random_state = 0, C = 1000, gamma = 0.0001, kernel = 'rbf')
clf.fit(X_train, y_train)
y_train_pred_svm = clf.predict(X_train)
y_test_pred_svm = clf.predict(X_test)
score = clf.score(X_test, y_test)
mse_train = mean_squared_error(y_train, y_train_pred_svm)
mse_test = mean_squared_error(y_test, y_test_pred_svm)
print('MSE Test: {}, MSE Train: {}' .format(mse_test, mse_train))
print("Accuracy score: {}\n".format(score))
print(classification_report(y_test, y_test_pred_svm, target_names=target_names))
scores['SVC'] = [score, metrics.precision_score(y_test, y_test_pred_svm), metrics.recall_score(y_test, y_test_pred_svm),
                      metrics.f1_score(y_test, y_test_pred_svm), mse_test, mse_train]

print('##################################')
print('NEURAL NETWORKS')
clf = MLPClassifier(solver='lbfgs', batch_size='auto', learning_rate='constant', power_t=0.5, max_iter=500, 
                    shuffle=True, tol=0.00001, verbose=True, 
                    early_stopping=False, validation_fraction=0.1,
                    hidden_layer_sizes=(5, ), activation='logistic', alpha=0.001, 
                    learning_rate_init=0.1, random_state=0)
clf.fit(X_train, y_train)
y_train_pred_nn = clf.predict(X_train)
y_test_pred_nn = clf.predict(X_test)
score = clf.score(X_test, y_test)
mse_train = mean_squared_error(y_train, y_train_pred_nn)
mse_test = mean_squared_error(y_test, y_test_pred_nn)
print('MSE Test: {}, MSE Train: {}' .format(mse_test, mse_train))
print("Accuracy score: {}\n".format(score))
print(classification_report(y_test, y_test_pred_nn, target_names=target_names))
scores['NN'] = [score, metrics.precision_score(y_test, y_test_pred_nn), metrics.recall_score(y_test, y_test_pred_nn),
                      metrics.f1_score(y_test, y_test_pred_nn), mse_test, mse_train]

print('##################################')
print('DECISION TREES')
clf = ExtraTreesClassifier(random_state=0, max_depth = 15, n_estimators = 100)
clf.fit(X_train, y_train)
y_train_pred_et = clf.predict(X_train)
y_test_pred_et = clf.predict(X_test)
score = clf.score(X_test, y_test)
mse_train = mean_squared_error(y_train, y_train_pred_et)
mse_test = mean_squared_error(y_test, y_test_pred_et)
print('MSE Test: {}, MSE Train: {}' .format(mse_test, mse_train))
print("Accuracy score: {}\n".format(score))
print(classification_report(y_test, y_test_pred_et, target_names=target_names))
scores['ET'] = [score, metrics.precision_score(y_test, y_test_pred_et), 
                         metrics.recall_score(y_test, y_test_pred_et), 
                         metrics.f1_score(y_test, y_test_pred_et), mse_test, mse_train]

print('##################################')
print('LOGISTIC REGRESSION')
clf = LogisticRegression(solver='lbfgs', random_state = 0, C = 10, max_iter = 100)
clf.fit(X_train, y_train)
y_train_pred_lr = clf.predict(X_train)
y_test_pred_lr = clf.predict(X_test)
score = clf.score(X_test, y_test)
mse_train = mean_squared_error(y_train, y_train_pred_lr)
mse_test = mean_squared_error(y_test, y_test_pred_lr)
print('MSE Test: {}, MSE Train: {}' .format(mse_test, mse_train))
print("Accuracy score: {}\n".format(score))
print(classification_report(y_test, y_test_pred_lr, target_names=target_names))
scores['LR'] = [score, metrics.precision_score(y_test, y_test_pred_lr), metrics.recall_score(y_test, y_test_pred_lr),
                      metrics.f1_score(y_test, y_test_pred_lr), mse_test, mse_train]

print('##################################')
print('RANDOM FOREST')
clf = RandomForestClassifier(random_state = 0, max_depth = 10, n_estimators = 100)
clf.fit(X_train, y_train)
y_train_pred_rf = clf.predict(X_train)
y_test_pred_rf = clf.predict(X_test)
score = clf.score(X_test, y_test)
mse_train = mean_squared_error(y_train, y_train_pred_rf)
mse_test = mean_squared_error(y_test, y_test_pred_rf)
print('MSE Test: {}, MSE Train: {}' .format(mse_test, mse_train))
print("Accuracy score: {}\n".format(score))
print(classification_report(y_test, y_test_pred_rf, target_names=target_names))
scores['RF'] = [score, metrics.precision_score(y_test, y_test_pred_rf), metrics.recall_score(y_test, y_test_pred_rf),
                      metrics.f1_score(y_test, y_test_pred_rf), mse_test, mse_train]

SUPPORT VECTOR MACHINES
MSE Test: 2.084507042253521, MSE Train: 1.591549295774648
Accuracy score: 0.4788732394366197

              precision    recall  f1-score   support

          -1       0.57      0.41      0.48        41
           1       0.41      0.57      0.48        30

    accuracy                           0.48        71
   macro avg       0.49      0.49      0.48        71
weighted avg       0.50      0.48      0.48        71

##################################
NEURAL NETWORKS
MSE Test: 1.295774647887324, MSE Train: 0.9295774647887324
Accuracy score: 0.676056338028169

              precision    recall  f1-score   support

          -1       0.70      0.76      0.73        41
           1       0.63      0.57      0.60        30

    accuracy                           0.68        71
   macro avg       0.67      0.66      0.66        71
weighted avg       0.67      0.68      0.67        71

##################################
DECISION TREES
MSE Test: 1.6338028169014085, MSE

In [62]:
scores_df = pd.DataFrame.from_dict(scores, orient='index')
scores_df.columns=['accuracy', 'precision', 'recall', 'f1-score', 'mse test', 'mse train']
scores_df

,accuracy,precision,recall,f1-score,mse test,mse train
SVC,0.478873,0.414634,0.566667,0.478873,2.084507,1.591549
NN,0.676056,0.629630,0.566667,0.596491,1.295775,0.929577
ET,0.591549,0.514286,0.600000,0.553846,1.633803,0.028169
LR,0.478873,0.400000,0.466667,0.430769,2.084507,1.619718
RF,0.619718,0.538462,0.700000,0.608696,1.521127,0.000000


In [63]:
test_df['PLabel NN'] = y_test_pred_nn
test_df['PLabel ET'] = y_test_pred_et
test_df['PLabel RF'] = y_test_pred_rf

test_feat = analysis_ml.filter(['NAME', 'Time Label'])
analysis = test_df.merge(test_feat, on = ['NAME'], how = 'left')
analysis = analysis.filter(['NAME', 'TIME W/ CUTS', 'TIME W/O CUTS', 
                            'NODES W/ CUTS', 'NODES W/O CUTS', 'PLabel NN', 'PLabel ET', 'PLabel RF', 'Time Label'])
analysis['Minimum Time'] = analysis[['TIME W/O CUTS', 'TIME W/ CUTS']].min(axis = 1)
analysis['Minimum Nodes'] = analysis[['NODES W/O CUTS', 'NODES W/ CUTS']].min(axis = 1)

analysis

,NAME,TIME W/ CUTS,TIME W/O CUTS,NODES W/ CUTS,NODES W/O CUTS,PLabel NN,PLabel ET,PLabel RF,Time Label,Minimum Time,Minimum Nodes
0,30n20b8.mps.gz,152.430964,147.722575,1617.0,13632.0,-1,1,1,-1,147.722575,1617.0
1,mspsphard01i.mps.gz,10493.451548,3408.665521,323371.0,152457.0,-1,-1,-1,-1,3408.665521,152457.0
2,rail507.mps.gz,183.736345,205.840109,836.0,1010.0,-1,1,-1,1,183.736345,836.0
3,neos-5193246-nerang.mps.gz,1171.813190,1525.663670,7082.0,17117.0,-1,1,1,1,1171.813190,7082.0
4,gen-ip002.mps.gz,1482.181497,1182.150688,5859548.0,5112787.0,-1,1,1,-1,1182.150688,5112787.0
...,...,...,...,...,...,...,...,...,...,...,...
66,neos-3611689-kaihu.mps.gz,1.927490,18.028007,21.0,29199.0,1,1,1,1,1.927490,21.0
67,neos-1601936.mps.gz,11527.267046,14945.885458,14529.0,1.0,1,-1,-1,1,11527.267046,1.0
68,neos-1425699.mps.gz,0.011907,0.017642,1.0,1.0,1,1,1,1,0.011907,1.0
69,neos-4322846-ryton.mps.gz,818.066564,670.952097,664.0,522.0,-1,-1,-1,-1,670.952097,522.0


In [64]:
rf_pred_time = []
rf_pred_nodes = []        
for index, row in analysis.iterrows():
    if row['PLabel RF'] == 1.0:
        rf_pred_time.append(row['TIME W/ CUTS'])
        rf_pred_nodes.append(row['NODES W/ CUTS'])
    elif row['PLabel RF'] == -1.0:
        rf_pred_time.append(row['TIME W/O CUTS'])
        rf_pred_nodes.append(row['NODES W/O CUTS'])

nn_pred_time = []
nn_pred_nodes = []        
for index, row in analysis.iterrows():
    if row['PLabel NN'] == 1.0:
        nn_pred_time.append(row['TIME W/ CUTS'])
        nn_pred_nodes.append(row['NODES W/ CUTS'])
    elif row['PLabel NN'] == -1.0:
        nn_pred_time.append(row['TIME W/O CUTS'])
        nn_pred_nodes.append(row['NODES W/O CUTS'])

et_pred_time = []
et_pred_nodes = []        
for index, row in analysis.iterrows():
    if row['PLabel ET'] == 1.0:
        et_pred_time.append(row['TIME W/ CUTS'])
        et_pred_nodes.append(row['NODES W/ CUTS'])
    elif row['PLabel ET'] == -1.0:
        et_pred_time.append(row['TIME W/O CUTS'])
        et_pred_nodes.append(row['NODES W/O CUTS'])

        
analysis['PTime RF'] = rf_pred_time
analysis['PTime NN'] = nn_pred_time
analysis['PTime ET'] = et_pred_time

analysis['PNodes RF'] = rf_pred_nodes
analysis['PNodes NN'] = nn_pred_nodes
analysis['PNodes ET'] = et_pred_nodes

analysis

,NAME,TIME W/ CUTS,TIME W/O CUTS,NODES W/ CUTS,NODES W/O CUTS,PLabel NN,PLabel ET,PLabel RF,Time Label,Minimum Time,Minimum Nodes,PTime RF,PTime NN,PTime ET,PNodes RF,PNodes NN,PNodes ET
0,30n20b8.mps.gz,152.430964,147.722575,1617.0,13632.0,-1,1,1,-1,147.722575,1617.0,152.430964,147.722575,152.430964,1617.0,13632.0,1617.0
1,mspsphard01i.mps.gz,10493.451548,3408.665521,323371.0,152457.0,-1,-1,-1,-1,3408.665521,152457.0,3408.665521,3408.665521,3408.665521,152457.0,152457.0,152457.0
2,rail507.mps.gz,183.736345,205.840109,836.0,1010.0,-1,1,-1,1,183.736345,836.0,205.840109,205.840109,183.736345,1010.0,1010.0,836.0
3,neos-5193246-nerang.mps.gz,1171.813190,1525.663670,7082.0,17117.0,-1,1,1,1,1171.813190,7082.0,1171.813190,1525.663670,1171.813190,7082.0,17117.0,7082.0
4,gen-ip002.mps.gz,1482.181497,1182.150688,5859548.0,5112787.0,-1,1,1,-1,1182.150688,5112787.0,1482.181497,1182.150688,1482.181497,5859548.0,5112787.0,5859548.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,neos-3611689-kaihu.mps.gz,1.927490,18.028007,21.0,29199.0,1,1,1,1,1.927490,21.0,1.927490,1.927490,1.927490,21.0,21.0,21.0
67,neos-1601936.mps.gz,11527.267046,14945.885458,14529.0,1.0,1,-1,-1,1,11527.267046,1.0,14945.885458,11527.267046,14945.885458,1.0,14529.0,1.0
68,neos-1425699.mps.gz,0.011907,0.017642,1.0,1.0,1,1,1,1,0.011907,1.0,0.011907,0.011907,0.011907,1.0,1.0,1.0
69,neos-4322846-ryton.mps.gz,818.066564,670.952097,664.0,522.0,-1,-1,-1,-1,670.952097,522.0,670.952097,670.952097,670.952097,522.0,522.0,522.0


In [65]:
testing = analysis.copy()

In [66]:
bb_classes = ['Geometric Mean']
num_bb_classes = len(bb_classes)

bucket_min = [0, 10, 100, 1000, 3600]
bucket_max = [18000, 18000, 18000, 18000, 18000]
num_buckets = len(bucket_min)
assert(len(bucket_max) == num_buckets)
bb_buckets = ['[' + str(bucket_min[j]) + ',' + str(bucket_max[j]) + ')' for j in range(num_buckets)]

bb_metrics = ['Gmean']

In [67]:
avg_bb_cols = pd.MultiIndex.from_product([['Time','Node'],
                                     ['Cut','No Cut', 'Perfect', 'RF', 'ET', 'NN']],
                                    names=['Parameter','Setting'])

bb_row_names = pd.MultiIndex.from_product(
    [bb_classes, bb_buckets],
    names=['Metric', 'Bucket'])

avg_bb_df = pd.DataFrame(
    index = bb_row_names,
    dtype = float)

avg_bb_df = pd.DataFrame(
    columns = avg_bb_cols,
    index = bb_row_names,
    dtype = float)

In [68]:
df_0 = analysis[(analysis['Minimum Time'] >= 0)]
df_10 = analysis[(analysis['Minimum Time'] >= 10)]
df_100 = analysis[(analysis['Minimum Time'] >= 100)]
df_1000 = analysis[(analysis['Minimum Time'] >= 1000)]
df_3600 = analysis[(analysis['Minimum Time'] >= 3600)]

In [69]:
SHIFT_TIME = 60
SHIFT_NODES = 1000

df = [df_0, df_10, df_100, df_1000, df_3600]
t_cuts = []
t_nocuts = []
t_perfect = []
t_rf = []
t_et = []
t_nn = []

n_cuts = []
n_nocuts = []
n_perfect = []
n_rf = []
n_et = []
n_nn = []

instances = []

for data in df:
    time_nocuts = data['TIME W/O CUTS'].tolist()
    time_cuts = data['TIME W/ CUTS'].tolist()
    time_perfect = data['Minimum Time'].tolist()
    time_rf = data['PTime RF'].tolist()
    time_et = data['PTime ET'].tolist()
    time_nn = data['PTime NN'].tolist()
    
    timeshift_nocuts = [i + SHIFT_TIME for i in time_nocuts]
    timeshift_cuts = [i + SHIFT_TIME for i in time_cuts]
    timeshift_perfect = [i + SHIFT_TIME for i in time_perfect]
    timeshift_rf = [i + SHIFT_TIME for i in time_rf]
    timeshift_et = [i + SHIFT_TIME for i in time_et]
    timeshift_nn = [i + SHIFT_TIME for i in time_nn]
    
    
    time_geom_nocuts = geometric_mean(timeshift_nocuts) - SHIFT_TIME
    time_geom_cuts = geometric_mean(timeshift_cuts) - SHIFT_TIME
    time_geom_perfect = geometric_mean(timeshift_perfect) - SHIFT_TIME
    time_geom_rf = geometric_mean(timeshift_rf) - SHIFT_TIME
    time_geom_et = geometric_mean(timeshift_et) - SHIFT_TIME
    time_geom_nn = geometric_mean(timeshift_nn) - SHIFT_TIME
    
    
    t_nocuts.append(time_geom_nocuts)
    t_cuts.append(time_geom_cuts)
    t_perfect.append(time_geom_perfect)
    t_rf.append(time_geom_rf)
    t_et.append(time_geom_et)
    t_nn.append(time_geom_nn)
    
    nodes_nocuts = data['NODES W/O CUTS'].tolist()
    nodes_cuts = data['NODES W/ CUTS'].tolist()
    nodes_perfect = data['Minimum Nodes'].tolist()
    nodes_rf = data['PNodes RF'].tolist()
    nodes_et = data['PNodes ET'].tolist()
    nodes_nn = data['PNodes NN'].tolist()
    
    nodeshift_nocuts = [i + SHIFT_NODES for i in nodes_nocuts]
    nodeshift_cuts = [i + SHIFT_NODES for i in nodes_cuts]
    nodeshift_perfect = [i + SHIFT_NODES for i in nodes_perfect]
    nodeshift_rf = [i + SHIFT_NODES for i in nodes_rf]
    nodeshift_et = [i + SHIFT_NODES for i in nodes_et]
    nodeshift_nn = [i + SHIFT_NODES for i in nodes_nn]
    
    nodes_geom_nocuts = geometric_mean(nodeshift_nocuts) - SHIFT_NODES
    nodes_geom_cuts = geometric_mean(nodeshift_cuts) - SHIFT_NODES
    nodes_geom_perfect = geometric_mean(nodeshift_perfect) - SHIFT_NODES
    nodes_geom_rf = geometric_mean(nodeshift_rf) - SHIFT_NODES
    nodes_geom_et = geometric_mean(nodeshift_et) - SHIFT_NODES
    nodes_geom_nn = geometric_mean(nodeshift_nn) - SHIFT_NODES
    
    n_nocuts.append(nodes_geom_nocuts)
    n_cuts.append(nodes_geom_cuts)
    n_perfect.append(nodes_geom_perfect)
    n_rf.append(nodes_geom_rf)
    n_et.append(nodes_geom_et)
    n_nn.append(nodes_geom_nn)
    
    instances.append(len(data))

In [70]:
avg_bb_df['Instances'] = instances

avg_bb_df[('Time', 'Cut')] = t_cuts
avg_bb_df[('Time', 'No Cut')] = t_nocuts

avg_bb_df[('Time', 'RF')] = t_rf
avg_bb_df[('Time', 'ET')] = t_et
avg_bb_df[('Time', 'NN')] = t_nn

avg_bb_df[('Time', 'Perfect')] = t_perfect

avg_bb_df[('Node', 'Cut')] = n_cuts
avg_bb_df[('Node', 'No Cut')] = n_nocuts

avg_bb_df[('Node', 'RF')] = n_rf
avg_bb_df[('Node', 'ET')] = n_et
avg_bb_df[('Node', 'NN')] = n_nn

avg_bb_df[('Node', 'Perfect')] = n_perfect

avg_bb_df

Parameter                           Time                             \
Setting                              Cut        No Cut      Perfect   
Metric         Bucket                                                 
Geometric Mean [0,18000)      274.255724    290.558223   206.461896   
               [10,18000)     526.420781    532.883431   375.169877   
               [100,18000)   1399.083172   1345.238645   994.239132   
               [1000,18000)  4338.893940   4069.153788  2767.531988   
               [3600,18000)  7878.133243  10486.841431  6283.006111   

Parameter                                                             \
Setting                               RF            ET            NN   
Metric         Bucket                                                  
Geometric Mean [0,18000)      236.906988    263.849854    252.361027   
               [10,18000)     440.505547    474.995900    450.840959   
               [100,18000)   1195.598505   1307.366588   1246.770288   
               [1000,18000)  3595.626592   4246.288315   4186.852026   
               [3600,18000)  9980.411465  10486.841431  10042.037617   

Parameter                             Node                               \
Setting                                Cut         No Cut       Perfect   
Metric         Bucket                                                     
Geometric Mean [0,18000)       5873.772645   12660.610956   5074.993594   
               [10,18000)     11162.501804   23205.981020   9343.101918   
               [100,18000)    27001.148124   49570.160383  21590.049117   
               [1000,18000)  131214.637044  290463.280989  93209.687204   
               [3600,18000)  127719.553802  320029.782193  80506.633729   

Parameter                                                                 \
Setting                                 RF             ET             NN   
Metric         Bucket                                                      
Geometric Mean [0,18000)       6369.279254    7962.207605    8499.247138   
               [10,18000)     12315.469366   14258.777494   15240.269545   
               [100,18000)    31400.857787   38694.988146   44404.811721   
               [1000,18000)  173808.318526  247545.143942  306634.728585   
               [3600,18000)  208260.741208  320029.782193  505987.081059   

Parameter                   Instances  
Setting                                
Metric         Bucket                  
Geometric Mean [0,18000)           71  
               [10,18000)          53  
               [100,18000)         33  
               [1000,18000)        16  
               [3600,18000)         6

In [71]:
imp_time_rf = []
imp_nodes_rf = []

for i, j in zip(t_rf, t_cuts):
    imp = ((i-j)/j)*100
    imp_time_rf.append(imp)

for i, j in zip(n_rf, n_cuts):
    imp = ((i - j)/j) * 100
    imp_nodes_rf.append(imp)

In [72]:
avg_bb_df[('Improvement (%)', 'Time')] = imp_time_rf
avg_bb_df[('Improvement (%)', 'Nodes')] = imp_nodes_rf

avg_bb_df

Parameter                           Time                             \
Setting                              Cut        No Cut      Perfect   
Metric         Bucket                                                 
Geometric Mean [0,18000)      274.255724    290.558223   206.461896   
               [10,18000)     526.420781    532.883431   375.169877   
               [100,18000)   1399.083172   1345.238645   994.239132   
               [1000,18000)  4338.893940   4069.153788  2767.531988   
               [3600,18000)  7878.133243  10486.841431  6283.006111   

Parameter                                                             \
Setting                               RF            ET            NN   
Metric         Bucket                                                  
Geometric Mean [0,18000)      236.906988    263.849854    252.361027   
               [10,18000)     440.505547    474.995900    450.840959   
               [100,18000)   1195.598505   1307.366588   1246.770288   
               [1000,18000)  3595.626592   4246.288315   4186.852026   
               [3600,18000)  9980.411465  10486.841431  10042.037617   

Parameter                             Node                               \
Setting                                Cut         No Cut       Perfect   
Metric         Bucket                                                     
Geometric Mean [0,18000)       5873.772645   12660.610956   5074.993594   
               [10,18000)     11162.501804   23205.981020   9343.101918   
               [100,18000)    27001.148124   49570.160383  21590.049117   
               [1000,18000)  131214.637044  290463.280989  93209.687204   
               [3600,18000)  127719.553802  320029.782193  80506.633729   

Parameter                                                                 \
Setting                                 RF             ET             NN   
Metric         Bucket                                                      
Geometric Mean [0,18000)       6369.279254    7962.207605    8499.247138   
               [10,18000)     12315.469366   14258.777494   15240.269545   
               [100,18000)    31400.857787   38694.988146   44404.811721   
               [1000,18000)  173808.318526  247545.143942  306634.728585   
               [3600,18000)  208260.741208  320029.782193  505987.081059   

Parameter                   Instances Improvement (%)             
Setting                                          Time      Nodes  
Metric         Bucket                                             
Geometric Mean [0,18000)           71      -13.618216   8.435917  
               [10,18000)          53      -16.320639  10.328935  
               [100,18000)         33      -14.544144  16.294528  
               [1000,18000)        16      -17.130341  32.461075  
               [3600,18000)         6       26.684979  63.060968

In [73]:
train_df

,NAME,RANDOM SEED,TIME W/ CUTS,NODES W/ CUTS,TIME W/O CUTS,NODES W/O CUTS,GAP W/ CUTS,GAP W/O CUTS,INITIAL LP,PRESOLVING TIME,bin_vars,int_vars,cont_vars,linsetpart_constr,linsetcov_constr,lincard_constr,lineqknap_constr,linbinpack_constr,linintknap_constr,linmixbin_constr,nonzeroes,obj_dyn,presol_vars,PLabel NN,PLabel ET,PLabel RF
298,qiu.mps.gz,12.4,68.293240,10652.0,42.874904,9207.0,0.0,0.0,-9.200187e+03,0.042668,0.057143,0.000000,0.942857,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.114094,0.003428,0.827942,6.733402,-1,-1,-1
247,neos-933638.mps.gz,12.4,14587.569504,983.0,1289.192938,64.0,0.0,0.0,-2.492000e+03,1.474001,0.883395,0.000000,0.116605,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.237443,0.000423,3.000000,9.092345,-1,-1,-1
309,rlp1.mps.gz,12.4,0.211836,51.0,0.216273,225.0,0.0,0.0,1.324444e+01,0.026762,0.976139,0.000000,0.023861,0.000000,0.000000,0.147059,0.0,0.000000,0.000000,0.661765,0.026668,0.000000,5.888878,1,1,1
336,supportcase4.mps.gz,12.4,3.806084,1.0,113.729586,160.0,0.0,0.0,0.000000e+00,0.307568,1.000000,0.000000,0.000000,0.000000,0.647282,0.000000,0.0,0.000000,0.000000,0.244627,0.001267,0.000000,7.667626,1,1,1
142,mzzv11.mps.gz,12.4,422.340278,1356.0,1489.909837,8880.0,0.0,0.0,-2.599800e+04,19.519242,0.975488,0.024512,0.000000,0.112959,0.002948,0.232446,0.0,0.000000,0.000000,0.000105,0.001384,2.397940,9.234057,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,sp98ic.mps.gz,12.4,8603.418640,141964.0,13259.130078,432282.0,0.0,0.0,4.442776e+08,1.902435,1.000000,0.000000,0.000000,0.000000,0.020606,0.000000,0.0,0.000000,0.000000,0.219394,0.035195,1.531250,9.295967,1,1,1
192,neos-3755335-nizao.mps.gz,12.4,16.354954,1.0,16.846205,1.0,0.0,0.0,0.000000e+00,15.067371,0.127656,0.000000,0.872344,0.002072,0.000000,0.000000,0.0,0.000108,0.000000,0.350567,0.000121,2.595280,10.619814,-1,1,1
117,lectsched-4-obj.mps.gz,12.4,5.974980,5.0,17.866144,1395.0,0.0,0.0,3.000000e+00,1.154996,0.970130,0.029870,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.094330,0.000000,0.000737,0.000000,8.023225,-1,1,1
47,chromaticindex128-5.mps.gz,12.4,84.350632,1316.0,1192.607684,100314.0,0.0,0.0,3.000000e+00,0.309220,1.000000,0.000000,0.000000,0.272641,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000434,0.000000,9.128696,-1,1,1


In [74]:
train_df['PLabel NN'] = y_train_pred_nn
train_df['PLabel ET'] = y_train_pred_et
train_df['PLabel RF'] = y_train_pred_rf
train_feat = analysis_ml.filter(['NAME', 'Time Label'])
analysis = train_df.merge(train_feat, on = ['NAME'], how = 'left')
analysis = analysis.filter(['NAME', 'TIME W/ CUTS', 'TIME W/O CUTS', 
                            'NODES W/ CUTS', 'NODES W/O CUTS', 'PLabel NN', 'PLabel ET', 'PLabel RF', 'Time Label'])
analysis['Minimum Time'] = analysis[['TIME W/O CUTS', 'TIME W/ CUTS']].min(axis = 1)
analysis['Minimum Nodes'] = analysis[['NODES W/O CUTS', 'NODES W/ CUTS']].min(axis = 1)

analysis

,NAME,TIME W/ CUTS,TIME W/O CUTS,NODES W/ CUTS,NODES W/O CUTS,PLabel NN,PLabel ET,PLabel RF,Time Label,Minimum Time,Minimum Nodes
0,qiu.mps.gz,68.293240,42.874904,10652.0,9207.0,-1,-1,-1,-1,42.874904,9207.0
1,neos-933638.mps.gz,14587.569504,1289.192938,983.0,64.0,-1,-1,-1,-1,1289.192938,64.0
2,rlp1.mps.gz,0.211836,0.216273,51.0,225.0,1,1,1,1,0.211836,51.0
3,supportcase4.mps.gz,3.806084,113.729586,1.0,160.0,1,1,1,1,3.806084,1.0
4,mzzv11.mps.gz,422.340278,1489.909837,1356.0,8880.0,1,1,1,1,422.340278,1356.0
...,...,...,...,...,...,...,...,...,...,...,...
279,sp98ic.mps.gz,8603.418640,13259.130078,141964.0,432282.0,1,1,1,1,8603.418640,141964.0
280,neos-3755335-nizao.mps.gz,16.354954,16.846205,1.0,1.0,-1,1,1,1,16.354954,1.0
281,lectsched-4-obj.mps.gz,5.974980,17.866144,5.0,1395.0,-1,1,1,1,5.974980,5.0
282,chromaticindex128-5.mps.gz,84.350632,1192.607684,1316.0,100314.0,-1,1,1,1,84.350632,1316.0


In [75]:
rf_pred_time = []
rf_pred_nodes = []        
for index, row in analysis.iterrows():
    if row['PLabel RF'] == 1.0:
        rf_pred_time.append(row['TIME W/ CUTS'])
        rf_pred_nodes.append(row['NODES W/ CUTS'])
    elif row['PLabel RF'] == -1.0:
        rf_pred_time.append(row['TIME W/O CUTS'])
        rf_pred_nodes.append(row['NODES W/O CUTS'])

nn_pred_time = []
nn_pred_nodes = []        
for index, row in analysis.iterrows():
    if row['PLabel NN'] == 1.0:
        nn_pred_time.append(row['TIME W/ CUTS'])
        nn_pred_nodes.append(row['NODES W/ CUTS'])
    elif row['PLabel NN'] == -1.0:
        nn_pred_time.append(row['TIME W/O CUTS'])
        nn_pred_nodes.append(row['NODES W/O CUTS'])

et_pred_time = []
et_pred_nodes = []        
for index, row in analysis.iterrows():
    if row['PLabel ET'] == 1.0:
        et_pred_time.append(row['TIME W/ CUTS'])
        et_pred_nodes.append(row['NODES W/ CUTS'])
    elif row['PLabel ET'] == -1.0:
        et_pred_time.append(row['TIME W/O CUTS'])
        et_pred_nodes.append(row['NODES W/O CUTS'])

        
analysis['PTime RF'] = rf_pred_time
analysis['PTime NN'] = nn_pred_time
analysis['PTime ET'] = et_pred_time

analysis['PNodes RF'] = rf_pred_nodes
analysis['PNodes NN'] = nn_pred_nodes
analysis['PNodes ET'] = et_pred_nodes

analysis

,NAME,TIME W/ CUTS,TIME W/O CUTS,NODES W/ CUTS,NODES W/O CUTS,PLabel NN,PLabel ET,PLabel RF,Time Label,Minimum Time,Minimum Nodes,PTime RF,PTime NN,PTime ET,PNodes RF,PNodes NN,PNodes ET
0,qiu.mps.gz,68.293240,42.874904,10652.0,9207.0,-1,-1,-1,-1,42.874904,9207.0,42.874904,42.874904,42.874904,9207.0,9207.0,9207.0
1,neos-933638.mps.gz,14587.569504,1289.192938,983.0,64.0,-1,-1,-1,-1,1289.192938,64.0,1289.192938,1289.192938,1289.192938,64.0,64.0,64.0
2,rlp1.mps.gz,0.211836,0.216273,51.0,225.0,1,1,1,1,0.211836,51.0,0.211836,0.211836,0.211836,51.0,51.0,51.0
3,supportcase4.mps.gz,3.806084,113.729586,1.0,160.0,1,1,1,1,3.806084,1.0,3.806084,3.806084,3.806084,1.0,1.0,1.0
4,mzzv11.mps.gz,422.340278,1489.909837,1356.0,8880.0,1,1,1,1,422.340278,1356.0,422.340278,422.340278,422.340278,1356.0,1356.0,1356.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,sp98ic.mps.gz,8603.418640,13259.130078,141964.0,432282.0,1,1,1,1,8603.418640,141964.0,8603.418640,8603.418640,8603.418640,141964.0,141964.0,141964.0
280,neos-3755335-nizao.mps.gz,16.354954,16.846205,1.0,1.0,-1,1,1,1,16.354954,1.0,16.354954,16.846205,16.354954,1.0,1.0,1.0
281,lectsched-4-obj.mps.gz,5.974980,17.866144,5.0,1395.0,-1,1,1,1,5.974980,5.0,5.974980,17.866144,5.974980,5.0,1395.0,5.0
282,chromaticindex128-5.mps.gz,84.350632,1192.607684,1316.0,100314.0,-1,1,1,1,84.350632,1316.0,84.350632,1192.607684,84.350632,1316.0,100314.0,1316.0


In [76]:
training = analysis.copy()

In [77]:
avg_bb_cols = pd.MultiIndex.from_product([['Time','Node'],
                                     ['Cut','No Cut', 'Perfect', 'RF', 'ET', 'NN']],
                                    names=['Parameter','Setting'])

bb_row_names = pd.MultiIndex.from_product(
    [bb_classes, bb_buckets],
    names=['Metric', 'Bucket'])

avg_bb_df = pd.DataFrame(
    index = bb_row_names,
    dtype = float)

avg_bb_df = pd.DataFrame(
    columns = avg_bb_cols,
    index = bb_row_names,
    dtype = float)

In [78]:
df_0 = analysis[(analysis['Minimum Time'] >= 0)]
df_10 = analysis[(analysis['Minimum Time'] >= 10)]
df_100 = analysis[(analysis['Minimum Time'] >= 100)]
df_1000 = analysis[(analysis['Minimum Time'] >= 1000)]
df_3600 = analysis[(analysis['Minimum Time'] >= 3600)]

In [79]:
df = [df_0, df_10, df_100, df_1000, df_3600]
t_cuts = []
t_nocuts = []
t_perfect = []
t_rf = []
t_et = []
t_nn = []

n_cuts = []
n_nocuts = []
n_perfect = []
n_rf = []
n_et = []
n_nn = []

instances = []

for data in df:
    time_nocuts = data['TIME W/O CUTS'].tolist()
    time_cuts = data['TIME W/ CUTS'].tolist()
    time_perfect = data['Minimum Time'].tolist()
    time_rf = data['PTime RF'].tolist()
    time_et = data['PTime ET'].tolist()
    time_nn = data['PTime NN'].tolist()
    
    timeshift_nocuts = [i + SHIFT_TIME for i in time_nocuts]
    timeshift_cuts = [i + SHIFT_TIME for i in time_cuts]
    timeshift_perfect = [i + SHIFT_TIME for i in time_perfect]
    timeshift_rf = [i + SHIFT_TIME for i in time_rf]
    timeshift_et = [i + SHIFT_TIME for i in time_et]
    timeshift_nn = [i + SHIFT_TIME for i in time_nn]
    
    
    time_geom_nocuts = geometric_mean(timeshift_nocuts) - SHIFT_TIME
    time_geom_cuts = geometric_mean(timeshift_cuts) - SHIFT_TIME
    time_geom_perfect = geometric_mean(timeshift_perfect) - SHIFT_TIME
    time_geom_rf = geometric_mean(timeshift_rf) - SHIFT_TIME
    time_geom_et = geometric_mean(timeshift_et) - SHIFT_TIME
    time_geom_nn = geometric_mean(timeshift_nn) - SHIFT_TIME
    
    
    t_nocuts.append(time_geom_nocuts)
    t_cuts.append(time_geom_cuts)
    t_perfect.append(time_geom_perfect)
    t_rf.append(time_geom_rf)
    t_et.append(time_geom_et)
    t_nn.append(time_geom_nn)
    
    nodes_nocuts = data['NODES W/O CUTS'].tolist()
    nodes_cuts = data['NODES W/ CUTS'].tolist()
    nodes_perfect = data['Minimum Nodes'].tolist()
    nodes_rf = data['PNodes RF'].tolist()
    nodes_et = data['PNodes ET'].tolist()
    nodes_nn = data['PNodes NN'].tolist()
    
    nodeshift_nocuts = [i + SHIFT_NODES for i in nodes_nocuts]
    nodeshift_cuts = [i + SHIFT_NODES for i in nodes_cuts]
    nodeshift_perfect = [i + SHIFT_NODES for i in nodes_perfect]
    nodeshift_rf = [i + SHIFT_NODES for i in nodes_rf]
    nodeshift_et = [i + SHIFT_NODES for i in nodes_et]
    nodeshift_nn = [i + SHIFT_NODES for i in nodes_nn]
    
    nodes_geom_nocuts = geometric_mean(nodeshift_nocuts) - SHIFT_NODES
    nodes_geom_cuts = geometric_mean(nodeshift_cuts) - SHIFT_NODES
    nodes_geom_perfect = geometric_mean(nodeshift_perfect) - SHIFT_NODES
    nodes_geom_rf = geometric_mean(nodeshift_rf) - SHIFT_NODES
    nodes_geom_et = geometric_mean(nodeshift_et) - SHIFT_NODES
    nodes_geom_nn = geometric_mean(nodeshift_nn) - SHIFT_NODES
    
    n_nocuts.append(nodes_geom_nocuts)
    n_cuts.append(nodes_geom_cuts)
    n_perfect.append(nodes_geom_perfect)
    n_rf.append(nodes_geom_rf)
    n_et.append(nodes_geom_et)
    n_nn.append(nodes_geom_nn)
    
    instances.append(len(data))

In [80]:
avg_bb_df['Instances'] = instances

avg_bb_df[('Time', 'Cut')] = t_cuts
avg_bb_df[('Time', 'No Cut')] = t_nocuts

avg_bb_df[('Time', 'RF')] = t_rf
avg_bb_df[('Time', 'ET')] = t_et
avg_bb_df[('Time', 'NN')] = t_nn

avg_bb_df[('Time', 'Perfect')] = t_perfect

avg_bb_df[('Node', 'Cut')] = n_cuts
avg_bb_df[('Node', 'No Cut')] = n_nocuts

avg_bb_df[('Node', 'RF')] = n_rf
avg_bb_df[('Node', 'ET')] = n_et
avg_bb_df[('Node', 'NN')] = n_nn

avg_bb_df[('Node', 'Perfect')] = n_perfect

avg_bb_df

Parameter                           Time                            \
Setting                              Cut       No Cut      Perfect   
Metric         Bucket                                                
Geometric Mean [0,18000)      180.289030   234.354764   140.932538   
               [10,18000)     335.035041   423.271365   252.018916   
               [100,18000)    812.276847   961.606928   597.039947   
               [1000,18000)  3163.825972  3720.055473  2318.990813   
               [3600,18000)  7998.252381  9128.856992  7315.044662   

Parameter                                                           \
Setting                               RF           ET           NN   
Metric         Bucket                                                
Geometric Mean [0,18000)      140.932538   141.022853   170.125234   
               [10,18000)     252.018916   252.170485   297.635298   
               [100,18000)    597.039947   597.561841   685.698235   
               [1000,18000)  2318.990813  2318.990813  2556.542009   
               [3600,18000)  7315.044662  7315.044662  8647.894780   

Parameter                            Node                               \
Setting                               Cut         No Cut       Perfect   
Metric         Bucket                                                    
Geometric Mean [0,18000)      2610.230290    7071.275598   2308.375949   
               [10,18000)     4644.936216   12084.888121   4005.942797   
               [100,18000)    8084.358488   17483.568842   6970.149364   
               [1000,18000)  30596.520451   63843.349231  25495.225961   
               [3600,18000)  71905.735126  131109.440239  68851.825194   

Parameter                                                               \
Setting                                RF            ET             NN   
Metric         Bucket                                                    
Geometric Mean [0,18000)      2558.581747   2558.064016    3613.148716   
               [10,18000)     4463.640657   4463.224820    5957.008960   
               [100,18000)    8007.253033   8006.132256    9767.047459   
               [1000,18000)  27789.671830  27789.671830   33425.564324   
               [3600,18000)  84513.407387  84513.407387  114011.391481   

Parameter                   Instances  
Setting                                
Metric         Bucket                  
Geometric Mean [0,18000)          284  
               [10,18000)         206  
               [100,18000)        126  
               [1000,18000)        38  
               [3600,18000)         8

In [81]:
imp_time_rf = []
imp_nodes_rf = []

for i, j in zip(t_rf, t_cuts):
    imp = ((i-j)/j)*100
    imp_time_rf.append(imp)

for i, j in zip(n_rf, n_cuts):
    imp = ((i - j)/j) * 100
    imp_nodes_rf.append(imp)

In [82]:
avg_bb_df[('Improvement (%)', 'Time')] = imp_time_rf
avg_bb_df[('Improvement (%)', 'Nodes')] = imp_nodes_rf

avg_bb_df

Parameter                           Time                            \
Setting                              Cut       No Cut      Perfect   
Metric         Bucket                                                
Geometric Mean [0,18000)      180.289030   234.354764   140.932538   
               [10,18000)     335.035041   423.271365   252.018916   
               [100,18000)    812.276847   961.606928   597.039947   
               [1000,18000)  3163.825972  3720.055473  2318.990813   
               [3600,18000)  7998.252381  9128.856992  7315.044662   

Parameter                                                           \
Setting                               RF           ET           NN   
Metric         Bucket                                                
Geometric Mean [0,18000)      140.932538   141.022853   170.125234   
               [10,18000)     252.018916   252.170485   297.635298   
               [100,18000)    597.039947   597.561841   685.698235   
               [1000,18000)  2318.990813  2318.990813  2556.542009   
               [3600,18000)  7315.044662  7315.044662  8647.894780   

Parameter                            Node                               \
Setting                               Cut         No Cut       Perfect   
Metric         Bucket                                                    
Geometric Mean [0,18000)      2610.230290    7071.275598   2308.375949   
               [10,18000)     4644.936216   12084.888121   4005.942797   
               [100,18000)    8084.358488   17483.568842   6970.149364   
               [1000,18000)  30596.520451   63843.349231  25495.225961   
               [3600,18000)  71905.735126  131109.440239  68851.825194   

Parameter                                                               \
Setting                                RF            ET             NN   
Metric         Bucket                                                    
Geometric Mean [0,18000)      2558.581747   2558.064016    3613.148716   
               [10,18000)     4463.640657   4463.224820    5957.008960   
               [100,18000)    8007.253033   8006.132256    9767.047459   
               [1000,18000)  27789.671830  27789.671830   33425.564324   
               [3600,18000)  84513.407387  84513.407387  114011.391481   

Parameter                   Instances Improvement (%)             
Setting                                          Time      Nodes  
Metric         Bucket                                             
Geometric Mean [0,18000)          284      -21.829665  -1.978697  
               [10,18000)         206      -24.778341  -3.903080  
               [100,18000)        126      -26.497973  -0.953761  
               [1000,18000)        38      -26.702959  -9.173751  
               [3600,18000)         8       -8.541962  17.533612

In [83]:
df = [training, testing]

t_cuts = []
t_nocuts = []
t_perfect = []
t_rf = []
t_et = []
t_nn = []

n_cuts = []
n_nocuts = []
n_perfect = []
n_rf = []
n_et = []
n_nn = []

instances = []

for data in df:
    time_nocuts = data['TIME W/O CUTS'].tolist()
    time_cuts = data['TIME W/ CUTS'].tolist()
    time_perfect = data['Minimum Time'].tolist()
    time_rf = data['PTime RF'].tolist()
    time_et = data['PTime ET'].tolist()
    time_nn = data['PTime NN'].tolist()
    
    timeshift_nocuts = [i + SHIFT_TIME for i in time_nocuts]
    timeshift_cuts = [i + SHIFT_TIME for i in time_cuts]
    timeshift_perfect = [i + SHIFT_TIME for i in time_perfect]
    timeshift_rf = [i + SHIFT_TIME for i in time_rf]
    timeshift_et = [i + SHIFT_TIME for i in time_et]
    timeshift_nn = [i + SHIFT_TIME for i in time_nn]
    
    
    time_geom_nocuts = geometric_mean(timeshift_nocuts) - SHIFT_TIME
    time_geom_cuts = geometric_mean(timeshift_cuts) - SHIFT_TIME
    time_geom_perfect = geometric_mean(timeshift_perfect) - SHIFT_TIME
    time_geom_rf = geometric_mean(timeshift_rf) - SHIFT_TIME
    time_geom_et = geometric_mean(timeshift_et) - SHIFT_TIME
    time_geom_nn = geometric_mean(timeshift_nn) - SHIFT_TIME
    
    
    t_nocuts.append(time_geom_nocuts)
    t_cuts.append(time_geom_cuts)
    t_perfect.append(time_geom_perfect)
    t_rf.append(time_geom_rf)
    t_et.append(time_geom_et)
    t_nn.append(time_geom_nn)
    
    nodes_nocuts = data['NODES W/O CUTS'].tolist()
    nodes_cuts = data['NODES W/ CUTS'].tolist()
    nodes_perfect = data['Minimum Nodes'].tolist()
    nodes_rf = data['PNodes RF'].tolist()
    nodes_et = data['PNodes ET'].tolist()
    nodes_nn = data['PNodes NN'].tolist()
    
    nodeshift_nocuts = [i + SHIFT_NODES for i in nodes_nocuts]
    nodeshift_cuts = [i + SHIFT_NODES for i in nodes_cuts]
    nodeshift_perfect = [i + SHIFT_NODES for i in nodes_perfect]
    nodeshift_rf = [i + SHIFT_NODES for i in nodes_rf]
    nodeshift_et = [i + SHIFT_NODES for i in nodes_et]
    nodeshift_nn = [i + SHIFT_NODES for i in nodes_nn]
    
    nodes_geom_nocuts = geometric_mean(nodeshift_nocuts) - SHIFT_NODES
    nodes_geom_cuts = geometric_mean(nodeshift_cuts) - SHIFT_NODES
    nodes_geom_perfect = geometric_mean(nodeshift_perfect) - SHIFT_NODES
    nodes_geom_rf = geometric_mean(nodeshift_rf) - SHIFT_NODES
    nodes_geom_et = geometric_mean(nodeshift_et) - SHIFT_NODES
    nodes_geom_nn = geometric_mean(nodeshift_nn) - SHIFT_NODES
    
    n_nocuts.append(nodes_geom_nocuts)
    n_cuts.append(nodes_geom_cuts)
    n_perfect.append(nodes_geom_perfect)
    n_rf.append(nodes_geom_rf)
    n_et.append(nodes_geom_et)
    n_nn.append(nodes_geom_nn)
    
    instances.append(len(data))

In [84]:
train_test_classes = ['Geometric Mean']
train_test_sets = ['Train Set', 'Test Set']

In [85]:
train_test_cols = pd.MultiIndex.from_product([['Time','Node'],
                                     ['Cut', 'No Cut','Perfect', 'RF', 'ET', 'NN']],
                                    names=['Parameter','Setting'])

train_test_row_names = pd.MultiIndex.from_product(
    [train_test_classes, train_test_sets],
    names=['Metric', 'Set'])

train_test_df = pd.DataFrame(
    index = train_test_row_names,
    dtype = float)

train_test_df = pd.DataFrame(
    columns = train_test_cols,
    index = train_test_row_names,
    dtype = float)

In [86]:
train_test_df['Instances'] = instances

train_test_df[('Time', 'Cut')] = t_cuts
train_test_df[('Time', 'No Cut')] = t_nocuts
train_test_df[('Time', 'RF')] = t_rf
train_test_df[('Time', 'ET')] = t_et
train_test_df[('Time', 'NN')] = t_nn

train_test_df[('Time', 'Perfect')] = t_perfect

train_test_df[('Node', 'Cut')] = n_cuts
train_test_df[('Node', 'No Cut')] = n_nocuts
train_test_df[('Node', 'RF')] = n_rf
train_test_df[('Node', 'ET')] = n_et
train_test_df[('Node', 'NN')] = n_nn

train_test_df[('Node', 'Perfect')] = n_perfect

train_test_df

Parameter                       Time                                      \
Setting                          Cut      No Cut     Perfect          RF   
Metric         Set                                                         
Geometric Mean Train Set  180.289030  234.354764  140.932538  140.932538   
               Test Set   274.255724  290.558223  206.461896  236.906988   

Parameter                                                Node                \
Setting                           ET          NN          Cut        No Cut   
Metric         Set                                                            
Geometric Mean Train Set  141.022853  170.125234  2610.230290   7071.275598   
               Test Set   263.849854  252.361027  5873.772645  12660.610956   

Parameter                                                                     \
Setting                       Perfect           RF           ET           NN   
Metric         Set                                                             
Geometric Mean Train Set  2308.375949  2558.581747  2558.064016  3613.148716   
               Test Set   5074.993594  6369.279254  7962.207605  8499.247138   

Parameter                Instances  
Setting                             
Metric         Set                  
Geometric Mean Train Set       284  
               Test Set         71

In [87]:
best_time = []
best_nodes = []

zip_time = zip(t_rf, t_et, t_nn)
zip_time = list(zip_time)

zip_nodes = zip(n_rf, n_et, n_nn)
zip_nodes = list(zip_nodes)

for i in zip_time:
    best_time.append(min(i))

for i in zip_nodes:
    best_nodes.append(min(i))

In [88]:
imp_time = []
imp_nodes = []

for i, j in zip(best_time, t_cuts):
    imp = ((i - j)/j) * 100
    imp_time.append(imp)
    
for i, j in zip(best_nodes, n_cuts):
    imp = ((i - j)/j) * 100
    imp_nodes.append(imp)

In [89]:
train_test_df[('Improvement (%)', 'Time')] = imp_time
train_test_df[('Improvement (%)', 'Nodes')] = imp_nodes

train_test_df

Parameter                       Time                                      \
Setting                          Cut      No Cut     Perfect          RF   
Metric         Set                                                         
Geometric Mean Train Set  180.289030  234.354764  140.932538  140.932538   
               Test Set   274.255724  290.558223  206.461896  236.906988   

Parameter                                                Node                \
Setting                           ET          NN          Cut        No Cut   
Metric         Set                                                            
Geometric Mean Train Set  141.022853  170.125234  2610.230290   7071.275598   
               Test Set   263.849854  252.361027  5873.772645  12660.610956   

Parameter                                                                     \
Setting                       Perfect           RF           ET           NN   
Metric         Set                                                             
Geometric Mean Train Set  2308.375949  2558.581747  2558.064016  3613.148716   
               Test Set   5074.993594  6369.279254  7962.207605  8499.247138   

Parameter                Instances Improvement (%)            
Setting                                       Time     Nodes  
Metric         Set                                            
Geometric Mean Train Set       284      -21.829665 -1.998531  
               Test Set         71      -13.618216  8.435917